- All non-negative coefficients
- Bounds for Intercept depend on no_intercept flag also
- Specific order for some coefficients
- Min percentage by which the coefficients (of ordered features) differ

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy.optimize import lsq_linear

# User inputs

In [2]:
df = pd.read_excel("data/benchmark_data.xlsx", sheet_name="hc_data")

In [3]:
no_intercept = True

In [4]:
target = 'TotalWageCost_Values'

In [5]:
# Features in the expected ascending order of coefficients
features = ['HeadCount_DS_1', 'HeadCount_DS_3', 'HeadCount_Sr_DS_2', 'HeadCount_Sr_DS_1', 'HeadCount_DS_2']
features

['HeadCount_DS_1',
 'HeadCount_DS_3',
 'HeadCount_Sr_DS_2',
 'HeadCount_Sr_DS_1',
 'HeadCount_DS_2']

In [6]:
# Min percentage gaps between successive (ordered) features
min_gap_pct = [None, 0.13, 0.51, 0.09, 0.03]

# Constraints

In [50]:
# Initialize coefficients
len_coeffs = len(features) + 1
coeffs = list(np.zeros(len_coeffs))
print("Initialized coefficients:", coeffs)

Initialized coefficients: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [51]:
# Put constraints 
min_con_orig = [0, 56, 64, 108, 97, 111]
max_con_orig = [np.inf, 95, 106, 171, 160, 176]

if no_intercept:
    min_con_orig[0] = 0
    max_con_orig[0] = 0.0001

In [52]:
min_con = min_con_orig.copy()
min_con[0] = max(0, min_con[0])
min_con[1] = max(0, min_con[1])
min_con[2] = max(0, min_con[2] - (1+min_gap_pct[1])*max_con_orig[1])
min_con[3] = max(0, min_con[3] - (1+min_gap_pct[2])*max_con_orig[2])
min_con[4] = max(0, min_con[4] - (1+min_gap_pct[3])*max_con_orig[3])
min_con[5] = max(0, min_con[5] - (1+min_gap_pct[4])*max_con_orig[4])

print("Minimum constraints:", min_con)

Minimum constraints: [0, 56, 0, 0, 0, 0]


In [53]:
max_con = max_con_orig.copy()
max_con[0] = max(min_con[0] + 0.0001, max_con[0])
max_con[1] = max(min_con[1] + 0.0001, max_con[1])
max_con[2] = max(min_con[2] + 0.0001, max_con[2] - (1+min_gap_pct[1])*min_con_orig[1])
max_con[3] = max(min_con[3] + 0.0001, max_con[3] - (1+min_gap_pct[2])*min_con_orig[2])
max_con[4] = max(min_con[4] + 0.0001, max_con[4] - (1+min_gap_pct[3])*min_con_orig[3])
max_con[5] = max(min_con[5] + 0.0001, max_con[5] - (1+min_gap_pct[4])*min_con_orig[4])

print("Maximum constraints:", max_con)

Maximum constraints: [0.0001, 95, 42.720000000000006, 74.36, 42.27999999999999, 76.09]


# Model

y = X0 + X1*DS_1 + (1.13*X1+X2)*DS_3 + (1.51*(1.13*X1+X2)+X3)*Sr_DS_2 + (1.09*(1.51*(1.13*X1+X2)+X3)+X4)*Sr_DS_1 + (1.03*(1.09*(1.51*(1.13*X1+X2)+X3)+X4)+X5)*DS_2

y = X0 + X1*(DS_1+1.13*DS_3+1.51*1.13*Sr_DS_2+1.09*1.51*1.13*Sr_DS_1+1.03*1.09*1.51*1.13*DS_2) + X2*(DS_3+1.51*Sr_DS_2+1.09*1.51*Sr_DS_1+1.03*1.09*1.51*DS_2) + X3*(Sr_DS_2+1.09*Sr_DS_1+1.03*1.09*DS_2) + X4*(Sr_DS_1+1.03*DS_2) + X5*DS_2

In [54]:
# Feature engineer   
X = df[features].copy()
X['F1'] = X[features[0]] + (1+min_gap_pct[1])*X[features[1]] + (1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[2]] + (1+min_gap_pct[3])*(1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[3]] + (1+min_gap_pct[4])*(1+min_gap_pct[3])*(1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[4]]
X['F2'] = X[features[1]] + (1+min_gap_pct[2])*X[features[2]] + (1+min_gap_pct[3])*(1+min_gap_pct[2])*X[features[3]] + (1+min_gap_pct[4])*(1+min_gap_pct[3])*(1+min_gap_pct[2])*X[features[4]]
X['F3'] = X[features[2]] + (1+min_gap_pct[3])*X[features[3]] + (1+min_gap_pct[4])*(1+min_gap_pct[3])*X[features[4]]
X['F4'] = X[features[3]] + (1+min_gap_pct[4])*X[features[4]]
X['F5'] = X[features[4]]
X = X.drop(features, axis=1)

In [55]:
# Convert independent variables to a matrix
X = X.values

# Add an array of ones to act as intercept coefficient
ones = np.ones(X.shape[0])
# Combine array of ones and indepedent variables
X = np.concatenate((ones[:, np.newaxis], X), axis=1)
X

array([[  1.        ,  40.54859402,  27.034154  ,  12.6054    ,
          6.06      ,   2.        ],
       [  1.        ,  32.93676401,  21.182977  ,   9.3927    ,
          4.03      ,   1.        ],
       [  1.        ,  48.16042403,  32.885331  ,  15.8181    ,
          8.09      ,   3.        ],
       [  1.        ,  40.54859402,  27.034154  ,  12.6054    ,
          6.06      ,   2.        ],
       [  1.        ,  55.77225404,  38.736508  ,  19.0308    ,
         10.12      ,   4.        ],
       [  1.        ,  48.16042403,  32.885331  ,  15.8181    ,
          8.09      ,   3.        ],
       [  1.        ,  63.38408405,  44.587685  ,  22.2435    ,
         12.15      ,   5.        ],
       [  1.        ,  55.77225404,  38.736508  ,  19.0308    ,
         10.12      ,   4.        ],
       [  1.        ,  70.99591406,  50.438862  ,  25.4562    ,
         14.18      ,   6.        ],
       [  1.        ,  63.38408405,  44.587685  ,  22.2435    ,
         12.15      ,   5. 

In [56]:
# Convert target variable to a matrix
y = df[target].values
y

array([3107, 2538, 3647, 3107, 4243, 3647, 4828, 4243, 5391, 4828, 5965,
       5391, 6575, 5965, 7108, 6575, 7724, 7108])

In [57]:
# Run optimization
results = lsq_linear(X, y, bounds=(min_con, max_con), lsmr_tol='auto')
print("Results:\n", results)

Results:
  active_mask: array([ 1,  0, -1, -1, -1, -1])
        cost: 2761.242155821166
         fun: array([-24.10749773, -33.83170438,  14.61670893, -24.10749773,
        -2.65908442,  14.61670893,  -8.93487776,  -2.65908442,
         6.7893289 ,  -8.93487776,  11.51353555,   6.7893289 ,
       -19.76225779,  11.51353555,  25.96194886, -19.76225779,
       -11.31384448,  25.96194886])
     message: 'The relative change of the cost function is less than `tol`.'
         nit: 13
  optimality: 8.730271155960767e-09
      status: 2
     success: True
           x: array([1.00000000e-04, 7.60295758e+01, 1.35314634e-31, 2.38641979e-30,
       9.21827109e-30, 3.96587494e-30])


In [58]:
if results.success:
    # Transform the coefficients back to the context of original features 
    coeffs[0] = results.x[0]
    coeffs[1] = results.x[1]
    coeffs[2] = (1+min_gap_pct[1])*results.x[1] + results.x[2]
    coeffs[3] = (1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]
    coeffs[4] = (1+min_gap_pct[3])*((1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]) + results.x[4]
    coeffs[5] = (1+min_gap_pct[4])*((1+min_gap_pct[3])*((1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]) + results.x[4]) + results.x[5]
    print("Final Coefficients (including intercept):", coeffs)
else:
    print("Convergence was not achieved!")

Final Coefficients (including intercept): [9.999999999999999e-05, 76.02957579127217, 85.91342064413755, 129.7292651726477, 141.40489903818602, 145.6470460093316]
